Install library cryptography: pip install cryptography

In [1]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import os

In [2]:
# Fungsi untuk mengenkripsi teks
def encrypt(plaintext, key):
    iv = os.urandom(16)  # Generate IV (16 byte) # untuk generate kunci (data awal yang digenerate random)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend()) # cipher digunakan untuk encryption, menggunakan algoritma AES
    # AES untuk generate key/kunci nya

    # Padding untuk memastikan ukuran blok 16 byte
    padder = padding.PKCS7(128).padder() # 1 blok = 16 byte, 16 x 8 = 128 bit. akan padding block sebanyak 128 bit
    padded_data = padder.update(plaintext.encode()) + padder.finalize()

    encryptor = cipher.encryptor() # data akan diencrypt dengan fungsi encryptor.
    ciphertext = encryptor.update(padded_data) + encryptor.finalize() # dihasilkan chipertext dengan menambahkan dengan encryptor

    return iv + ciphertext  # Gabungkan IV dengan ciphertext

In [3]:
# Fungsi untuk mendekripsi teks
def decrypt(ciphertext, key):
    iv = ciphertext[:16]  # Ambil IV dari awal ciphertext
    encrypted_data = ciphertext[16:]

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend()) # pakai key yang sama (symmetric)

    decryptor = cipher.decryptor() # pakai function decryptor untuk menterjemahkan kembali.
    decrypted_padded = decryptor.update(encrypted_data) + decryptor.finalize()

    # Hapus padding
    unpadder = padding.PKCS7(128).unpadder() # unpadd untuk menghapus kode yang ditambahkan di bagian belakang pada bagian sebelumnya.
    plaintext = unpadder.update(decrypted_padded) + unpadder.finalize()

    return plaintext.decode()

In [4]:
# Kunci AES 256-bit (32 byte)
key = os.urandom(32) # key dibuat dengan 32 byte = 256 bit (semakin besar semakin bagus)
# print(key)

# Input teks yang akan dienkripsi
plaintext = " " # 'IN' panjangnya tetap sama, dibuat/dibulatkan jadi panjang datanya mendekati ke 16.

# Enkripsi
ciphertext = encrypt(plaintext, key)
print(f"Ciphertext: {ciphertext.hex()}")

# Dekripsi
decrypted_text = decrypt(ciphertext, key)
print(f"Decrypted Text: {decrypted_text}")

Ciphertext: 75394e17f8960e60d1c5fe12c07012639c687e5e4a0e6fe2dd30758ba9ca3b78
Decrypted Text:  


1. Jelaskan perbedaan peran algoritma AES,  mode CBC dan padding
2. Apakah yang digunakan kriptografi asimetris atau simetris jelaskan
3. Pada fungsi decrypt gantilah menjadi iv=os.urandom(16), apa yang terjadi jelaskan
4. Pada isi pesan plaintext, cobalah dari 1 huruf, 2 huruf dst. Lihatlah perubahan panjang chipertext, analisis perbedaannya
5. Bagaimana praktis pengamanan'key' agar isi pesan tidak mudah diketahui [implementasi industrinya]

jawaban:

1. AES (Advanced Encryption Standard) adalah algoritma enkripsi simetris yang digunakan untuk mengamankan data. AES memiliki ukuran kunci 128-bit, 192-bit, atau 256-bit.Mode CBC (Cipher Block Chaining) bekerja dengan cara setiap blok ciphertext tergantung pada blok plaintext sebelumnya yang telah dienkripsi. Ini memberikan keamanan lebih dibandingkan ECB tetapi tidak mendukung paralelisasi. Padding digunakan karena AES bekerja dalam ukuran blok tetap (16 byte per blok). Jika data tidak cukup panjang, maka ditambahkan padding agar sesuai dengan ukuran blok.

2. Kriptografi simetris menggunakan satu kunci untuk enkripsi dan dekripsi. Ini lebih cepat tetapi membutuhkan distribusi kunci yang aman. Sedangkan kriptografi asimetris menggunakan dua kunci (publik dan privat) seperti pada RSA dan ECC. Ini lebih aman untuk komunikasi antar pengguna tetapi lebih lambat dibanding simetris. Dalam hal ini, AES menggunakan kriptografi simetris, sedangkan komunikasi yang lebih aman biasanya menggunakan kriptografi dengan kombinasi simetris dan asimetris.

3. Jika diganti dengan menggunakan iv=os.urandom(16), maka hasil dekripsi akan tidak valid dan tidak akan terbaca, karena IV yang digunakan saat enkripsi dan dekripsi tidak cocok saat dibandingkan.

4. Dalam AES dengan mode CBC, panjang ciphertext bergantung pada panjang plaintext serta padding.
Jika plaintext hanya 1 huruf, padding akan memenuhi hingga 16 byte. Jika plaintext ditambah menjadi 17 huruf, blok tambahan akan dibuat. Dalam hal ini, plaintext akan berpengaruh pada perubahan ciphertext juga.

5. Implementasi industri: menggunakan Key Management System (KMS) seperti AWS, KMS atau Google Cloud KMS. Ada juga key yang disimpan di Hardware Security Module (HSM), bukan di dalam kode program. Ada juga yang menggunakan enkripsi kunci (Key Wrapping), di mana kunci utama dienkripsi dengan kunci lain yang lebih aman. Selain itu, ada juga yang menggunakan Secure Multi-Party Computation (SMPC) agar tidak ada satu pihak yang memiliki akses penuh ke kunci.

In [5]:
import time
# Fungsi enkripsi dengan berbagai mode
def encrypt_aes(plaintext, key, mode):
    iv = os.urandom(16)
    if mode == "CBC":
        cipher_mode = modes.CBC(iv)
    elif mode == "CFB":
        cipher_mode = modes.CFB(iv)
    elif mode == "GCM": # metode GCM untuk mendeteksi autentikasi data/integritas data terganggu atau tidak.
        cipher_mode = modes.GCM(iv)
    else:
        raise ValueError("Mode tidak didukung")

    cipher = Cipher(algorithms.AES(key), cipher_mode, backend=default_backend())
    plaintext_bytes = plaintext.encode()

    # Padding hanya diperlukan untuk CBC
    if mode == "CBC":
        padder = padding.PKCS7(128).padder()
        plaintext_bytes  = padder.update(plaintext_bytes)+ padder.finalize()

    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext_bytes) + encryptor.finalize()

    return iv + ciphertext

# Uji performa mode AES
key = os.urandom(32)
plaintext = "Data rahasia untuk eksperimen yang lebih kompleks."

for mode in ["CBC", "CFB", "GCM"]:
    start_time = time.time()
    plaintext = "Data rahasia untuk eksperimen yang lebih kompleks."
    ciphertext = encrypt_aes(plaintext, key, mode)
    end_time = time.time()
    print(f"Mode {mode} - Waktu Enkripsi: {end_time - start_time:.6f} detik")
    print(f"Ciphertext ({mode}): {ciphertext.hex()[:50]}...\n")

Mode CBC - Waktu Enkripsi: 0.000000 detik
Ciphertext (CBC): f42af6205dd00a0526bd72b156964ed7c7d4516043eb569618...

Mode CFB - Waktu Enkripsi: 0.001990 detik
Ciphertext (CFB): 5304f3331e997aebe336380c1742e2b6497cb7b9afe39d0e16...

Mode GCM - Waktu Enkripsi: 0.000997 detik
Ciphertext (GCM): 5a2da5fb1ff075831a772c2cf120a8c0ab36555dc8dd6fd2a0...



6. Apa kegunaan dan perbedaan masing-masing mode AES ini?

Jawab:

6. CBC cocok untuk file & database encryption, tapi tidak mendukung parallel processing. Sedangkan CFB cocok untuk stream encryption & secure communication (seperti TLS, VPN, SSH). Selain itu, mode GCM adalah mode yang paling aman dan cepat, yang cocok untuk network security protocols seperti TLS, IPsec & aplikasi modern.

In [6]:
from itertools import product
import string

# Fungsi brute force terhadap kunci pendek
def brute_force_attack(target_ciphertext, correct_key):
    charset = string.ascii_letters + string.digits
    key_length = len(correct_key)
    print(f"Memulai brute force pada kunci {key_length} karakter...")

    for attempt in product(charset, repeat=key_length):
        guessed_key = "".join(attempt).encode()
        if guessed_key == correct_key:
            print(f"Kunci ditemukan: {guessed_key.decode()}")
            break

# Simulasi kunci pendek (4 karakter)
correct_key = b"abcdefg"
correct_key = b"abc1" # panjang key nya hanya 4, makanya bisa ketemu dalam waktu yang cepat
# key harus menggunakan standar dari NIST, AES 256 bit. 

ciphertext = encrypt_aes("Tes brute force", correct_key.ljust(32, b'\x00'), "CBC")

# Menjalankan brute force
brute_force_attack(ciphertext, correct_key)


Memulai brute force pada kunci 4 karakter...
Kunci ditemukan: abc1


7. Cobalah berbagai variasi key dan key yang lebih panjang, amati prilakunya dan jelaskan mengapa AES direkomendasikan menggunakan panjang key 256 bit

Jawab:

7. AES direkomendasikan menggunakan panjang key 256 bit supaya keamanannya tetap terjaga. Semakin panjang bit dari key nya maka akan semakin susah pula key untuk diretas/dihack.

In [7]:
# Fungsi Enkripsi AES-GCM
def encrypt_gcm(plaintext, key):
    iv = os.urandom(12)  # IV 12-byte untuk GCM
    cipher = Cipher(algorithms.AES(key), modes.GCM(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    tag = encryptor.tag  # Authentication tag untuk verifikasi

    return iv, ciphertext, tag  # Mengembalikan IV, Ciphertext, dan Tag

# Fungsi Dekripsi AES-GCM
def decrypt_gcm(iv, ciphertext, tag, key):
    cipher = Cipher(algorithms.AES(key), modes.GCM(iv, tag), backend=default_backend())
    decryptor = cipher.decryptor()

    try:
        decrypted_text = decryptor.update(ciphertext) + decryptor.finalize()
        #print(f"decript:{decrypted_text.hex()}")
        return decrypted_text.decode()
    except Exception as e:
        return f"[GCM DETECTED CHANGE] Dekripsi gagal: {str(e)}"

# Buat kunci AES-256 (32 byte)
key = os.urandom(32)
plaintext = "Pesan rahasia yang sangat penting"

# Enkripsi dengan AES-GCM
iv, ciphertext, tag = encrypt_gcm(plaintext, key) # tag yang membuat GCM bisa mendeteksi serangan yang bisa mengubah data.
print(f"Ciphertext (GCM): {ciphertext.hex()}\n")

# Dekripsi Tanpa Perubahan
decrypted_text = decrypt_gcm(iv, ciphertext, tag, key)
print(f" Dekripsi Sukses: {decrypted_text}\n")

# Mengubah Satu Byte di Ciphertext untuk (Simulasi Serangan)
ciphertext_modified = bytearray(ciphertext)
ciphertext_modified[5] ^= 0xFF  # Membalikkan satu byte pada index ke-5 (0xFF = 256)

# Coba Dekripsi dengan Ciphertext yang Telah Diubah
decrypted_text_modified = decrypt_gcm(iv, bytes(ciphertext_modified), tag, key)
print(f" Dekripsi Gagal Setelah Perubahan: {decrypted_text_modified}\n")


Ciphertext (GCM): 451de90dced617ce5986f05a83b8c49f47b6bd0f6383d7722ceaa56fd95a7178d3

 Dekripsi Sukses: Pesan rahasia yang sangat penting

 Dekripsi Gagal Setelah Perubahan: [GCM DETECTED CHANGE] Dekripsi gagal: 



In [8]:
# Fungsi Enkripsi AES-CBC dengan Padding PKCS7
def encrypt_cbc(plaintext, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Gunakan padding PKCS7 agar panjang plaintext menjadi kelipatan 16 byte
    padder = padding.PKCS7(128).padder()
    padded_plaintext = padder.update(plaintext.encode()) + padder.finalize()

    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    return ciphertext

# Fungsi Dekripsi AES-CBC dengan Unpadding PKCS7
def decrypt_cbc(iv, ciphertext, key):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    
    try:
        decrypted_padded = decryptor.update(ciphertext) + decryptor.finalize()
        print(f"decript:{decrypted_padded.hex()}")

        # Hapus padding setelah dekripsi
        unpadder = padding.PKCS7(128).unpadder()
        decrypted_text = unpadder.update(decrypted_padded) + unpadder.finalize()
        
        return decrypted_text.decode()
    
    except ValueError as e:
        return f"[CBC DETECTED ERROR] Dekripsi gagal: {str(e)}"

# Enkripsi dengan AES-CBC
key = os.urandom(32)  # AES-256 key
iv_cbc = os.urandom(16)  # IV untuk CBC
plaintext = "Pesan penting yang harus dirahasiakan"

ciphertext_cbc = encrypt_cbc(plaintext, key, iv_cbc)
print(f"Ciphertext (CBC): {ciphertext_cbc.hex()}\n") # isinya decrypt text yang isinya masih ada padding

# Modifikasi Satu Byte dalam Ciphertext (Simulasi Serangan)
ciphertext_cbc_modified = bytearray(ciphertext_cbc)

ciphertext_cbc_modified[5] ^= 0xFF  # Mengubah satu byte

# Dekripsi Ciphertext yang Telah Diubah
decrypted_cbc_modified = decrypt_cbc(iv_cbc, bytes(ciphertext_cbc_modified), key)
print(f" CBC Dekripsi Setelah Perubahan: {decrypted_cbc_modified}\n")


Ciphertext (CBC): 2828628d3051b857f19e08f00f26d379fe6e7a503d153f6698d73de58d03598b787026cc2841744a107caa5cea172a5c

decript:c27e4f9d256788b4421339aaec0c85ca6e672068618d7573206469726168617369616b616e0b0b0b0b0b0b0b0b0b0b0b
 CBC Dekripsi Setelah Perubahan: [CBC DETECTED ERROR] Dekripsi gagal: 'utf-8' codec can't decode byte 0xc2 in position 0: invalid continuation byte



8. Bagaimana perbedaan respon model CBC dan mode GCM terhadap serangan?

Jawab:

8. Kedua model tersebut memiliki perbedaan utama dalam aspek keamanan dan efisiensi. Model CBC rentan terhadap serangan padding oracle jika sistem memberikan informasi tentang padding yang salah saat dekripsi, serta dapat dimanipulasi dengan serangan bit-flipping jika tidak ada autentikasi tambahan. Selain itu, model CBC tidak memiliki mekanisme bawaan untuk mencegah serangan chosen-ciphertext dan replay attack, yang memungkinkan penyerang dapat mengubah atau menggunakan kembali ciphertext tanpa terdeteksi sama sekali. Akan tetapi dari sisi efisiensinya, CBC lebih lambat karena prosesnya bergantung pada blok sebelumnya sehingga tidak bisa diproses secara paralel. Sementara itu, model GCM lebih aman dan efisien karena memiliki mekanisme Authenticated Encryption with Associated Data (AEAD) yang dapat melindungi data dari serangan modifikasi sehingga integritas sebuah data tetap terjaga. GCM juga lebih cepat dibanding CBC karena berbasis Counter Mode (CTR), yang memungkinkan enkripsi dapat diproses secara paralel. Selain itu, penggunaan nonce unik dalam GCM mencegah serangan replay membuatnya menjadi lebih baik dari segi keamanan maupun performanya.